In [ ]:
from __future__ import division
import random
import cv2
import glob
import os
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import preprocessing
from matplotlib import pyplot as plt

In [ ]:
def load_data(filename):
    f = open(filename, 'r')
    csv_r = csv.reader(f, delimiter='\t')
    data = []
    for row in csv_r:
        data.append([float(elem) for elem in row])
    return  np.array(data)

In [ ]:
def svc_param_selection(X, y, nfolds):

    Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
    gammas = [0.001, 0.01, 0.1, 1, 100]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(svm.SVC(kernel='rbf'), param_grid, cv=nfolds, n_jobs = 8, pre_dispatch= 16, verbose = 5 )
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search.best_params_

In [ ]:
# Load training data: data is already normalized
X = load_data('X_train.txt')
tot_pixel = len(X)
print(X.shape)

# load labels
y = load_data(('y_train.txt'))
y = np.ravel(y)
print(y.shape)

In [ ]:
# Select a balanced portion of vessel/background pixels to tune parameters
random.seed(21)

all_idx = list(range(len(X)))
idx_ves = list(np.ravel(np.where(y == 1)))
print(len(idx_ves))

idx_back = list(set(all_idx)-set(idx_ves))
print(len(idx_back))

print(idx_ves[0:10])
print(idx_back[0:10])


In [ ]:
# Select 10% vessel pixels and 10% backgroud pixels for tuning
ves_tuning = random.sample(idx_ves, 24843 ) 
back_tuning = random.sample(idx_back, 24843)
idx_tuning = ves_tuning + back_tuning


In [ ]:
# Cross validation to tune parameters
best_parameters = svc_param_selection(X[idx_tuning,:], y[idx_tuning], 3)

In [ ]:
best_parameters

In [ ]:
clf = svm.SVC(gamma=1, C=1000)
clf.fit(X[idx_tuning, :], y[idx_tuning])  

In [ ]:
# save the model to disk
filename = 'svm_model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [ ]:
#Load test 
# Load Data
X = load_data('X_test.txt')
print(X.shape)
print(len(X))

y = load_data(('y_test.txt'))
y = np.ravel(y)
print(y.shape)

In [ ]:
#load the model from disk
loaded_model = pickle.load(open('svm_model.sav', 'rb'))
result = loaded_model.score(X, y)
print(result)
predict = loaded_model.predict(X)
print(predict)

In [ ]:
# Create output folder
output = 'output_svm'
os.mkdir(output)

In [ ]:
# Retrieve name of images 
list_name = list()
images_path = '../../filter/filter_test'
for infile in glob.glob(images_path + '/*.png'):
    p, im_name = os.path.split(infile)
    file, ext = os.path.splitext(im_name)
    list_name.append(file)
    
    
j = 0    
for num, i in enumerate(range(0,len(predict), 8281)):
    im =  np.asarray(predict[j:i+8281])
    im = im.reshape((91,91))
    cv2.imwrite(output + '/' + list_name[num] + '.png' , im*255)
    j = i+8281